In [ ]:
import pandas as pd

In [ ]:
orders = pd.read_csv('ecommerce_data/orders_dataset.csv')

In [ ]:
orders.head(5)

In [ ]:
order_items = pd.read_csv('ecommerce_data/order_items_dataset.csv')

In [ ]:
order_items.head(5)

In [ ]:
order_items.describe()

In [ ]:
products = pd.read_csv('ecommerce_data/products_dataset.csv')

In [ ]:
products.head(5)

In [ ]:
products['product_id'].nunique()

In [ ]:
products['product_category_name'].nunique()

In [ ]:
products['product_category_name'].unique()

In [ ]:
customers = pd.read_csv('ecommerce_data/customers_dataset.csv')

In [ ]:
customers.head(5)

In [ ]:
reviews = pd.read_csv('ecommerce_data/order_reviews_dataset.csv')

In [ ]:
reviews.head(5)

In [ ]:
# How much revenue did we generate in 2023 compared to 2022?

In [ ]:
order_items.head(2)

In [ ]:
orders.head(2)

In [ ]:
sales_data = pd.merge(left = order_items[['order_id','order_item_id', 'product_id', 'price']],
         right = orders[['order_id', 'order_status', 'order_purchase_timestamp', 'order_delivered_customer_date']],
         on = 'order_id')

In [ ]:
sales_data.head(2)

In [ ]:
sales_data['order_status'].unique()

In [ ]:
sales_delivered = sales_data[sales_data['order_status'] == 'delivered']

In [ ]:
sales_delivered['order_purchase_timestamp'] = pd.to_datetime(sales_delivered['order_purchase_timestamp'])

In [ ]:
sales_delivered['month'] = sales_delivered['order_purchase_timestamp'].apply(lambda timestamp:timestamp.month)

In [ ]:
sales_delivered['year'] = sales_delivered['order_purchase_timestamp'].apply(lambda timestamp:timestamp.year)

In [ ]:
sales_delivered

In [ ]:
sales_delivered_2023 = sales_delivered[sales_delivered['year']==2023]
sales_delivered_2022 = sales_delivered[sales_delivered['year']==2022]

In [ ]:
#total revenue in 2023
print(f"total_revenue in 2023: {sales_delivered_2023['price'].sum()}")

In [ ]:
# revenue compared to 2022
revenue_growth=(sales_delivered_2023['price'].sum()-sales_delivered_2022['price'].sum())/sales_delivered_2022['price'].sum()
print(f"{revenue_growth*100}%")

In [ ]:
# What is the month-over-month growth trend in 2023?

In [ ]:
monthly_growth = sales_delivered_2023[['price', 'month']].groupby('month')['price'].sum().pct_change()

In [ ]:
monthly_growth

In [ ]:
pct = monthly_growth.mean()

In [ ]:
print(f"Monthly growth in 2023: {100*pct}%")

In [ ]:
# average order value in 2023

In [ ]:
print(f"Average order value in 2023: {sales_delivered_2023.groupby('order_id')['price'].sum().mean()}")

In [ ]:
avg_order_2023 = sales_delivered_2023.groupby('order_id')['price'].sum().mean()
avg_order_2022 = sales_delivered_2022.groupby('order_id')['price'].sum().mean()

print(f"Compared to last year: {100*(avg_order_2023 - avg_order_2022)/(avg_order_2022)}%")

In [ ]:
# Total orders in 2023
print(f"Total number of orders in 2023: {sales_delivered_2023['order_id'].nunique()}")

In [ ]:
total_orders_2023 = sales_delivered_2023['order_id'].nunique()
total_orders_2022 = sales_delivered_2022['order_id'].nunique()

print(f"Compared to last year: {100*(total_orders_2023 - total_orders_2022)/(total_orders_2022)}%")

In [ ]:
# revenue trend in the last 12 months

In [ ]:
sales_delivered_2023.groupby(['year', 'month'])['price'].sum().reset_index().plot(x='month', y='price')

In [ ]:
# top product categories in the last 12 months

In [ ]:
products.head(2)

In [ ]:
sales_delivered_2023.head(2)

In [ ]:
sales_categories_2023 = pd.merge(left=products[['product_id', 'product_category_name']],
        right=sales_delivered_2023[['product_id', 'price']])

In [ ]:
product_sales = sales_categories_2023.groupby('product_category_name')['price'].sum().sort_values(ascending=False)

In [ ]:
product_sales.plot(kind='bar')

In [ ]:
#sales by state in the last 12 months

In [ ]:
sales_delivered_2023.head(2)

In [ ]:
orders.head(2)

In [ ]:
customers.head(2)

In [ ]:
sales_2023_customers = pd.merge(left= sales_delivered_2023[['order_id', 'price']],
         right= orders[['order_id', 'customer_id']])

In [ ]:
sales_states_2023 = pd.merge(left = sales_2023_customers, right=customers[['customer_id', 'customer_state']])

In [ ]:
sales_states_2023 = sales_states_2023.groupby('customer_state')['price'].sum().sort_values(ascending=False)

In [ ]:
sales_states_2023 = sales_states_2023.reset_index()

In [ ]:
import plotly.express as px
import pandas as pd

# Create choropleth map
fig = px.choropleth(
    sales_states_2023,
    locations='customer_state',
    color='price',
    locationmode='USA-states',
    scope='usa',
    title='Revenue by State',
    color_continuous_scale='Reds'
)

fig.show()

In [ ]:
reviews.head(2)

In [ ]:
sales_delivered_2023.head(2)

In [ ]:
sales_delivered_2023['order_delivered_customer_date'] = pd.to_datetime(sales_delivered_2023['order_delivered_customer_date'])

In [ ]:
sales_delivered_2023['delivery_speed'] = (sales_delivered_2023['order_delivered_customer_date']-sales_delivered_2023['order_purchase_timestamp']).apply(lambda t:t.days)

In [ ]:
sales_delivered_2023 = sales_delivered_2023.merge(reviews[['order_id','review_score']])

In [ ]:
review_speed = sales_delivered_2023[['order_id', 'delivery_speed', 'review_score']].drop_duplicates()

In [ ]:
review_speed.groupby('delivery_speed')['review_score'].mean().reset_index()

In [ ]:
def categorize_delivery_speed(days):
    if days<=3:
        return '1-3 days'
    if days<=7:
        return '4-7 days'
    return '8+ days'
review_speed['delivery_time'] = review_speed['delivery_speed'].apply(categorize_delivery_speed)

In [ ]:
# Avg Delivery Time
review_speed.groupby('delivery_time')['review_score'].mean().reset_index()

In [ ]:
# Review Score for 2023
review_speed['review_score'].mean()

In [ ]:
# Review Score
review_speed['review_score'].value_counts(normalize=True).plot(kind='barh')

In [ ]:
# Order status - 2023
orders['year'] = pd.to_datetime(orders['order_purchase_timestamp']).apply(lambda t:t.year)
orders[orders['year']==2023]['order_status'].value_counts(normalize=True)

In [ ]:
# average delivery time - 2023
review_speed['delivery_speed'].mean()